# Homework 5 (Full mark: 100pt)

You can use ``Google Colab`` if you would like to use GPU.
- https://colab.research.google.com/notebooks/welcome.ipynb
- https://theorydb.github.io/dev/2019/08/23/dev-ml-colab/

# 1. Regression (50pt)

**For this question, using PyTorch, implement the 1) ridge regression and 2) Lasso. You can refer to the tutorial link below for how to implement linear regression. Note that the ridge regression is the linear regression with L2 penalty, and the Lasso is the linear regression with L1 penalty. You should use ````Boston```` dataset as shown in the code below. You should not only write the code for the models, but also train them and show the test MSE.**
- https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/linear_regression/main.py

<div>
<img src="figures/regressions.png" width="700"/>
</div>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
INPUT_DIR='/content/drive/MyDrive/ML IE/HW5'
OUTPUT_DIR = '/content/drive/MyDrive/ML IE/HW5'

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
# To fix the random seed
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# load data
#boston = pd.read_csv('data/Boston.csv').drop('Unnamed: 0', axis=1)
boston = pd.read_csv('/content/drive/MyDrive/ML IE/HW5/data/Boston.csv').drop('Unnamed: 0', axis=1)
data = torch.FloatTensor(boston.values)
X = data[:,:-1] # Input (X)
y = data[:,-1].reshape(-1, 1) # Ground Truth (y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [12]:
num_epochs = 1000          #100    I change num_epochs and lr for effective train
learning_rate = 0.000001   #0.0003
torch.manual_seed(3)

class Ridge(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        ## Write your answer here
        self.input_dim = input_dim
        self.output_dim = 1
        self.fc1 = nn.Linear(self.input_dim,self.output_dim)

    def forward(self, X):
        ## Write your answer here
        y1 = self.fc1(X)

        return y1

class Lasso(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        ## Write your answer here
        self.input_dim = input_dim
        self.output_dim = 1
        self.fc1 = nn.Linear(self.input_dim,self.output_dim)

    def forward(self, X):
        ## Write your answer here
        y1 = self.fc1(X)
        return y1

## Write your answer here (Training code)
def train(model, X, y, epochs, c = 1, penalty= 'l1'):

    # regularity
    c = torch.FloatTensor([c])
    p = 1 if penalty == 'l1' else 2

    criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):

        inputs = X
        targets = y

        # Forward pass
        outputs = model(inputs)
        #print(p)
        loss = criterion(outputs, targets) + c * torch.norm(model.fc1.weight, p =p)  # total loss = mse + penalty(L1 or L2)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (epoch+1) % 5 == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


## Train Ridge and Lasso and show the final test MSE.
input_dim = X.size()[1]  #13 features

model_r = Ridge(input_dim)
model_l = Lasso(input_dim)
model1 = train(model_l, X_train, y_train, epochs = num_epochs, c=1, penalty= 'l2')
model2 = train(model_r, X_train, y_train, epochs = num_epochs, c=1, penalty= 'l1')


y_hat_r = model_r(X_test).detach().numpy()
y_hat_l = model_l(X_test).detach().numpy()
y_test_np = y_test.detach().numpy()
y_hat_l = model_l(X_test).detach().numpy()
final_MSE_r = np.mean((y_hat_r - y_test_np)**2)
final_MSE_l = np.mean((y_hat_l - y_test_np)**2)
print('final_MSE for ridge:',final_MSE_r)
print('final_MSE for lasso:',final_MSE_l)

Epoch [5/1000], Loss: 104.7677
Epoch [10/1000], Loss: 85.4990
Epoch [15/1000], Loss: 84.3764
Epoch [20/1000], Loss: 83.5967
Epoch [25/1000], Loss: 83.0311
Epoch [30/1000], Loss: 82.5997
Epoch [35/1000], Loss: 82.2531
Epoch [40/1000], Loss: 81.9605
Epoch [45/1000], Loss: 81.7027
Epoch [50/1000], Loss: 81.4676
Epoch [55/1000], Loss: 81.2478
Epoch [60/1000], Loss: 81.0388
Epoch [65/1000], Loss: 80.8374
Epoch [70/1000], Loss: 80.6421
Epoch [75/1000], Loss: 80.4515
Epoch [80/1000], Loss: 80.2649
Epoch [85/1000], Loss: 80.0818
Epoch [90/1000], Loss: 79.9020
Epoch [95/1000], Loss: 79.7252
Epoch [100/1000], Loss: 79.5513
Epoch [105/1000], Loss: 79.3802
Epoch [110/1000], Loss: 79.2118
Epoch [115/1000], Loss: 79.0459
Epoch [120/1000], Loss: 78.8827
Epoch [125/1000], Loss: 78.7219
Epoch [130/1000], Loss: 78.5636
Epoch [135/1000], Loss: 78.4077
Epoch [140/1000], Loss: 78.2541
Epoch [145/1000], Loss: 78.1029
Epoch [150/1000], Loss: 77.9540
Epoch [155/1000], Loss: 77.8074
Epoch [160/1000], Loss: 77.

# 2. Autoencoder (50pt)
**Autoencoder is an unsupervised neural network model for learning representations of the input. In the figure below, you can see the structure of an autoencoder network. Given the original input image, we first encode the image using ``Encoder`` to a compressed representation, and reconstruct the image by using ``Decoder`` given the compressed representation. The compressed represesntation can be used as the dimension reduced representation of the original input. In this regard, autoencoder is also known as a model for dimensionality reduction (Recall PCA was also a method for dimensionality reduction). Note that the encoder and the decoder can be any neural network model such as MLP, CNN, MLP, etc.**

**For this question, you will use MNIST dataset to implement two versions of autoencoders: 1) An MLP-based autoencoder, and 2) A CNN-based autoencoder. The figure below shows an example of the MLP-based autoencoder. For the MLP-based autoencoder, you should follow the structure of the autoencoder shown in the figure below. For the CNN-based autoencoder, you are free to choose the architecture. You only need to implement ``Autoencoder_MLP`` class and ``Autoencoder_CNN`` class. Note that for ``Autoencoder_MLP``, you should flatten the original image into a vector, whereas for ``Autoencoder_CNN``, you can use the original image without any modification. After implementing these two classes, save the figures and observe the results. Write a few sentences to describe your findings. You do not need to submit the saved figure and the dataset for your final submission.**

<div>
<img src="figures/autoencoder.png" width="700"/>
</div>

In [7]:
import os

import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image

# Make a directory "saved_img" if it does not exist
#if not os.path.exists('./saved_img'):
#    os.mkdir('./saved_img')

if not os.path.exists(INPUT_DIR + '/saved_img'):
    os.mkdir(INPUT_DIR + '/saved_img')


def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

#dataset = MNIST('./data', transform=img_transform)
dataset = MNIST(INPUT_DIR+'/data', transform=img_transform)  # use Google Colab
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class Autoencoder_MLP(nn.Module):
    def __init__(self):
        super(Autoencoder_MLP, self).__init__()
        ## Write your answer here
        self.encoder = nn.Sequential(
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12),
            nn.ReLU(),
            nn.Linear(12, 3)
                    )

        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(),
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28*28),
            nn.Sigmoid()
        )
    def forward(self, x):
        ## Write your answer here
        x = x.cuda()
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)

        return decoded


class Autoencoder_CNN(nn.Module):
    def __init__(self):
        super(Autoencoder_CNN, self).__init__()
        ## Write your answer here
        self.encoder = nn.Sequential(
                nn.Conv2d(1,5,5),
                nn.ReLU(),
                nn.MaxPool2d(2),
                nn.Conv2d(5, 10,10),
                nn.ReLU(),
                nn.MaxPool2d(2,2)
                    )

        self.decoder = nn.Sequential(
                nn.ConvTranspose2d(10,5,5),
                nn.ReLU(),
                nn.ConvTranspose2d(5, 1, 24),
                nn.Sigmoid()
        )
    def forward(self, x):
        ## Write your answer here
        x = x.cuda()
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)

        return decoded




# Uncomment below correspondingly

model = Autoencoder_MLP().cuda()
#model = Autoencoder_CNN().cuda()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data

        # Flatten the image for Autoencoder_MLP (You can remove this line for Autoencoder_CNN)
        img = img.view(img.size(0), -1)

        # forward pass
        output = model(img)
        #print(output.size())
        img = img.to(output.device)  #에러 메시지 때문에 추가하였습니다
        loss = criterion(output, img)
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print log and save images
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, num_epochs, loss.item()))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, INPUT_DIR+'/saved_img/image_{}.png'.format(epoch))

KeyboardInterrupt: ignored

Autoencoder_MLP 와 Autoencoder_CNN 에 의해 만들어진 이미지를 관찰한 결과 공통적으로 train을 거듭할수록 이미지 파일의 숫자가 더 선명해지는 것을 관찰할 수 있었다. 또한 두 모델을 비교했을 때 ,loss값에서나 이미지의 선명도 면에서 큰 차이는 없었다.